# Q1.3 - Data Preprocessing

### First we need to load the data

In [20]:
# Load data from parquet file
import pandas as pd
import numpy as np
from project_1.config import PROJ_ROOT, PROCESSED_DATA_DIR

sets_dict = {}
sets = ["a", "b", "c"]

for set_name in sets:
    directory = PROCESSED_DATA_DIR / f"set_{set_name}.parquet"
    temp_set = pd.read_parquet(directory)
    sets_dict[f"set_{set_name}"] = temp_set

# Assure the loading was correct
print(sets_dict["set_a"].shape)
sets_dict["set_a"].head(10)


(183416, 43)


,RecordID,Time,Age,BUN,Creatinine,GCS,Gender,Glucose,HCO3,HCT,...,PaCO2,PaO2,pH,DiasABP,MAP,SaO2,SysABP,Lactate,Cholesterol,TroponinI
0,132539.0,2025-03-10 00:00:00,54.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,132539.0,2025-03-10 01:00:00,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,132539.0,2025-03-10 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,132539.0,2025-03-10 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,132539.0,2025-03-10 04:00:00,NaN,NaN,NaN,15.0,NaN,NaN,NaN,33.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,132539.0,2025-03-10 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,132539.0,2025-03-10 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,132539.0,2025-03-10 08:00:00,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,132539.0,2025-03-10 09:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,132539.0,2025-03-10 10:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Step 0 - Outliers Removal

In [27]:
# Save's code for Outlier Removal

# After this cell, data/processed should include the following files:
# - set_a_cleaned.parquet
# - set_b_cleaned.parquet
# - set_c_cleaned.parquet

# Define the clean function
def clean_df(df):
    """
    Clean the input DataFrame according to the following rules:
      - Missing value handling: For Age, Gender, Height, ICUType, Weight, set -1 to NA.
      - Height outlier removal: Set Height to NA if < 100 cm or >= 300 cm.
      - Weight outlier removal: Set Weight to NA if < 20 kg or >= 300 kg.
      - PaO2 corrections: Set PaO2 equal to 0 to NA; if PaO2 equals 7.47, correct it to 74.7.
      - pH unit correction: If pH is between 65 and 80, divide by 10; if between 650 and 800, divide by 100.
      - Temperature corrections: Set Temp to NA if Temp is less than 20.
    """
    # 1. Missing value handling: Replace -1 with np.nan for selected columns.
    missing_cols = ["Age", "Gender", "Height", "ICUType", "Weight"]
    for col in missing_cols:
        df.loc[df[col] == -1, col] = np.nan

    # 2. Height outlier removal: Set Height to NA if < 100 or >= 300.
    df.loc[(df["Height"] < 100) | (df["Height"] >= 300), "Height"] = np.nan

    # 3. Weight outlier removal: Set Weight to NA if < 20 or >= 300.
    df.loc[(df["Weight"] < 20) | (df["Weight"] >= 300), "Weight"] = np.nan

    # 4. PaO2 corrections:
    #    Set PaO2 equal to 0 to NA, and if PaO2 is 7.47, correct it to 74.7.
    df.loc[df["PaO2"] == 0, "PaO2"] = np.nan
    df.loc[df["PaO2"] == 7.47, "PaO2"] = 74.7

    # 5. pH unit correction:
    #    If pH is between 65 and 80, divide by 10; if between 650 and 800, divide by 100.
    def correct_ph(ph):
        if pd.isna(ph):
            return ph
        if 65 <= ph <= 80:
            return ph / 10.0
        elif 650 <= ph <= 800:
            return ph / 100.0
        else:
            return ph
    df["pH"] = df["pH"].apply(correct_ph)

    # 6. Temperature corrections: Set Temp to NA if Temp is < 20.
    df.loc[df["Temp"] < 20, "Temp"] = np.nan

    return df

In [29]:
# Use the function to clean the data
for set_key, df in sets_dict.items():
    # Clean the DataFrame
    cleaned_df = clean_df(df)
    sets_dict[set_key] = cleaned_df  # Update dictionary (optional)

    # Export to Parquet file (e.g., "set_a_cleaned.parquet")
    output_filename = PROCESSED_DATA_DIR / f"{set_key}_cleaned.parquet"
    cleaned_df.to_parquet(output_filename, index=False)
    print(f"Cleaned data for {set_key} saved as {output_filename}")

# Optionally, print the shape and first few rows of one cleaned DataFrame
print(sets_dict["set_a"].shape)
sets_dict["set_a"].head(10)


Cleaned data for set_a saved as /Users/francescobondi/Desktop/stuff/ETH/FS25/ML for Healthcare/project-1-ml4hc/data/processed/set_a_cleaned.parquet
Cleaned data for set_b saved as /Users/francescobondi/Desktop/stuff/ETH/FS25/ML for Healthcare/project-1-ml4hc/data/processed/set_b_cleaned.parquet
Cleaned data for set_c saved as /Users/francescobondi/Desktop/stuff/ETH/FS25/ML for Healthcare/project-1-ml4hc/data/processed/set_c_cleaned.parquet
(183416, 43)


,RecordID,Time,Age,BUN,Creatinine,GCS,Gender,Glucose,HCO3,HCT,...,PaCO2,PaO2,pH,DiasABP,MAP,SaO2,SysABP,Lactate,Cholesterol,TroponinI
0,132539.0,2025-03-10 00:00:00,54.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,132539.0,2025-03-10 01:00:00,54.0,NaN,NaN,15.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,132539.0,2025-03-10 02:00:00,54.0,NaN,NaN,15.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,132539.0,2025-03-10 03:00:00,54.0,NaN,NaN,15.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,132539.0,2025-03-10 04:00:00,54.0,NaN,NaN,15.0,0.0,NaN,NaN,33.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,132539.0,2025-03-10 05:00:00,54.0,NaN,NaN,15.0,0.0,NaN,NaN,33.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,132539.0,2025-03-10 06:00:00,54.0,NaN,NaN,15.0,0.0,NaN,NaN,33.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,132539.0,2025-03-10 08:00:00,54.0,NaN,NaN,15.0,0.0,NaN,NaN,33.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,132539.0,2025-03-10 09:00:00,54.0,NaN,NaN,15.0,0.0,NaN,NaN,33.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,132539.0,2025-03-10 10:00:00,54.0,NaN,NaN,15.0,0.0,NaN,NaN,33.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Step 1 - Use Forward Filling imputation (use only Set A for now)

In [30]:
df = sets_dict["set_a"]

# Ensure the DataFrame is sorted by RecordID and Time
df.sort_values(by=["RecordID", "Time"], inplace=True)

# Get a list of all columns except "RecordID" and "Time"
other_cols = [col for col in df.columns if col != "RecordID" and col != "Time"]

# Group by RecordID and apply forward fill for each group.
df[other_cols] = df.groupby("RecordID")[other_cols].ffill()

# Display the first 10 rows to check the result
df.head(10)

,RecordID,Time,Age,BUN,Creatinine,GCS,Gender,Glucose,HCO3,HCT,...,PaCO2,PaO2,pH,DiasABP,MAP,SaO2,SysABP,Lactate,Cholesterol,TroponinI
0,132539.0,2025-03-10 00:00:00,54.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,132539.0,2025-03-10 01:00:00,54.0,NaN,NaN,15.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,132539.0,2025-03-10 02:00:00,54.0,NaN,NaN,15.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,132539.0,2025-03-10 03:00:00,54.0,NaN,NaN,15.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,132539.0,2025-03-10 04:00:00,54.0,NaN,NaN,15.0,0.0,NaN,NaN,33.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,132539.0,2025-03-10 05:00:00,54.0,NaN,NaN,15.0,0.0,NaN,NaN,33.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,132539.0,2025-03-10 06:00:00,54.0,NaN,NaN,15.0,0.0,NaN,NaN,33.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,132539.0,2025-03-10 08:00:00,54.0,NaN,NaN,15.0,0.0,NaN,NaN,33.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,132539.0,2025-03-10 09:00:00,54.0,NaN,NaN,15.0,0.0,NaN,NaN,33.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,132539.0,2025-03-10 10:00:00,54.0,NaN,NaN,15.0,0.0,NaN,NaN,33.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Success, the values are being forwarded. Keep in mind that this can be altered in the future

# Step 2 - Do Interpolation as backward filling method
### Maybe explain why?

In [31]:
# Ensure the "Time" column is in datetime format
df["Time"] = pd.to_datetime(df["Time"])

# Set "Time" as the DataFrame index for time-based interpolation
df = df.set_index("Time")

# Identify the columns to interpolate (exclude non-numeric columns like "RecordID")
cols_to_interp = [col for col in df.columns if col != "RecordID"]

# Apply time-based interpolation; limit_direction='both' fills NaNs at the start and end too.
df[cols_to_interp] = df[cols_to_interp].interpolate(method='time', limit_direction='both')

# If you want to restore the "Time" column as a regular column, reset the index
df = df.reset_index()

# Display the updated DataFrame
df.head(20)

,Time,RecordID,Age,BUN,Creatinine,GCS,Gender,Glucose,HCO3,HCT,...,PaCO2,PaO2,pH,DiasABP,MAP,SaO2,SysABP,Lactate,Cholesterol,TroponinI
0,2025-03-10 00:00:00,132539.0,54.0,15.0,2.9,15.0,0.0,89.0,23.0,36.40,...,36.0,258.0,7.48,47.00,87.000000,98.0,89.000000,3.60,152.0,3.5
1,2025-03-10 01:00:00,132539.0,54.0,17.0,1.0,15.0,0.0,147.0,23.0,29.60,...,43.0,385.0,7.40,50.00,96.500000,98.0,135.000000,3.25,160.0,0.9
2,2025-03-10 02:00:00,132539.0,54.0,20.0,0.8,15.0,0.0,210.0,27.0,29.80,...,35.0,195.0,7.32,69.50,55.000000,97.0,130.000000,6.20,158.0,1.1
3,2025-03-10 03:00:00,132539.0,54.0,8.0,0.7,15.0,0.0,75.0,21.0,26.75,...,62.0,285.0,7.36,47.00,85.000000,98.0,105.000000,6.40,169.0,31.0
4,2025-03-10 04:00:00,132539.0,54.0,9.0,0.3,15.0,0.0,154.0,20.0,33.70,...,45.0,431.0,7.47,78.00,85.750000,99.0,136.000000,2.30,111.0,7.4
5,2025-03-10 05:00:00,132539.0,54.0,20.0,0.7,15.0,0.0,124.0,21.0,33.70,...,31.0,128.0,7.42,68.00,70.666667,98.0,132.000000,1.60,152.0,11.7
6,2025-03-10 06:00:00,132539.0,54.0,10.0,2.9,15.0,0.0,222.0,22.0,33.70,...,43.0,477.0,7.32,61.00,76.000000,96.0,123.250000,6.60,92.0,0.4
7,2025-03-10 08:00:00,132539.0,54.0,47.0,1.1,15.0,0.0,52.0,25.0,33.70,...,39.0,69.0,7.34,54.25,91.000000,98.0,92.000000,1.50,195.0,1.1
8,2025-03-10 09:00:00,132539.0,54.0,169.0,0.6,15.0,0.0,137.0,29.0,33.70,...,42.0,101.0,7.38,78.00,78.000000,94.0,98.000000,1.20,84.0,1.4
9,2025-03-10 10:00:00,132539.0,54.0,17.0,0.5,15.0,0.0,96.0,20.0,33.70,...,35.0,151.0,7.30,59.00,64.000000,96.0,131.000000,1.50,101.0,1.1


Interpolation is especially appropriate in time series settings for several reasons:
	1.	Temporal Continuity:
Time series data are inherently ordered by time, and values are usually related to their adjacent measurements. Interpolation leverages this temporal continuity to estimate missing values based on the trend or pattern observed in neighboring time points.
	2.	Preserving Dynamics:
Methods such as time-based interpolation take into account the actual timestamps (or time intervals) between measurements. This means that the interpolation can reflect varying gaps between observations, ensuring that the estimates align more closely with the natural evolution of the underlying process.
	3.	Smooth Transitions:
In many time series, especially in healthcare data, the changes in values tend to be gradual rather than abrupt. Interpolation methods (like linear or time-based interpolation) are well-suited to capture these smooth transitions, leading to more realistic estimates compared to simply carrying forward the last observation (forward fill) or filling with a constant value.
	4.	Handling Irregular Spacing:
In cases where time series measurements are not evenly spaced, time-based interpolation can accommodate the actual time differences. This is particularly useful when there are missing values at the beginning or end of the series or when observations are recorded at variable intervals.
	5.	Enhancing Model Quality:
Many statistical and machine learning models require a complete dataset. Using interpolation to fill missing values ensures that the temporal structure is maintained and that the data remain consistent with the observed trends, which can improve model performance and reliability.

In summary, interpolation is an effective technique in time series settings because it respects the temporal order and dynamics of the data, provides smooth and realistic estimates for missing values, and helps maintain the integrity of the dataset for subsequent analysis or modeling.

# Step 3 - Scale the data

In [25]:
from sklearn.preprocessing import StandardScaler, RobustScaler

# Get the columns to scale
cols_to_scale = [col for col in df.columns if col not in ["RecordID", "Time", "Gender"]]

# Define scaler
### However, we will need to use a different scaler for each column, based on whether it is normally-distributed or not
### For normally-distributed columns, we use the StandardScaler
### For non-normally-distributed columns, we use the RobustScaler

#! AGE COULD BE DONE WITH MIN-MAX
nd_cols = ["Height", "Weight", "Age", "Albumin", "Cholesterol", "DiasABP", "HCO3", "HCT", "HR", "Mg", "MAP", "Na", "NIDiasABP", "NIMAP", "NISysABP", "SysABP", "PaCO2", "PaO2", "Platelets", "RespRate", "Temp"]
nnd_cols = [col for col in cols_to_scale if col not in nd_cols]

scaler_nd = StandardScaler()
scaler_nnd = RobustScaler()

# Fit and then transform the scaler
scaled_values_nd = scaler_nd.fit_transform(df[nd_cols])
scaled_values_nnd = scaler_nnd.fit_transform(df[nnd_cols])

# Convert the scaled arrays to DataFrames (preserve the original index)
df_scaled_nd = pd.DataFrame(scaled_values_nd, columns=nd_cols, index=df.index)
df_scaled_nnd = pd.DataFrame(scaled_values_nnd, columns=nnd_cols, index=df.index)

# Combine both scaled DataFrames along the columns axis
df_scaled = pd.concat([df_scaled_nd, df_scaled_nnd], axis=1)

# Option 2: If you only want to include a selected subset of unmodified columns along with the scaled columns, use:
df_final = pd.concat([df[["RecordID", "Time", "Gender"]], df_scaled], axis=1)

df_final.head(10)



,RecordID,Time,Gender,Height,Weight,Age,Albumin,Cholesterol,DiasABP,HCO3,...,ALP,ALT,AST,Bilirubin,FiO2,MechVent,pH,SaO2,Lactate,TroponinI
0,132539.0,2025-03-10 00:00:00,0.0,-1.060851,-2.575945,-0.596332,1.671639,-0.013487,-0.832594,-0.109176,...,0.132075,-0.176471,0.450704,1.545455,2.5,0.0,1.125,0.0,1.384615,0.285714
1,132539.0,2025-03-10 01:00:00,0.0,-1.060851,-2.575945,-0.596332,1.967793,0.172112,-0.608431,-0.109176,...,0.000000,-0.294118,0.436620,0.000000,1.5,0.0,0.125,0.0,1.115385,-0.126984
2,132539.0,2025-03-10 02:00:00,0.0,-1.060851,-2.575945,-0.596332,-1.734132,0.125712,0.848629,0.830987,...,0.773585,-0.205882,-0.380282,0.181818,0.0,0.0,-0.875,-0.5,3.384615,-0.095238
3,132539.0,2025-03-10 03:00:00,0.0,-1.060851,-2.575945,-0.596332,1.523562,0.380910,-0.832594,-0.579257,...,-0.698113,-0.588235,1.126761,-0.181818,0.0,0.0,-0.375,0.0,3.538462,4.650794
4,132539.0,2025-03-10 04:00:00,0.0,-1.060851,-2.575945,-0.596332,0.487023,-0.964680,1.483758,-0.814297,...,-0.490566,-0.558824,-0.225352,0.363636,0.5,0.0,1.000,0.5,0.384615,0.904762
5,132539.0,2025-03-10 05:00:00,0.0,-1.060851,-2.575945,-0.596332,0.042792,-0.013487,0.736548,-0.579257,...,0.509434,5.323529,35.577465,24.909091,2.5,0.0,0.375,0.0,-0.153846,1.587302
6,132539.0,2025-03-10 06:00:00,0.0,-1.060851,-2.575945,-0.596332,0.635100,-1.405476,0.213500,-0.344216,...,-0.452830,-0.088235,25.661972,-0.454545,2.5,0.0,-0.875,-1.0,3.692308,-0.206349
7,132539.0,2025-03-10 08:00:00,0.0,-1.060851,-2.575945,-0.596332,0.635100,0.984105,-0.290867,0.360906,...,1.584906,0.058824,1.450704,-0.090909,0.0,0.0,-0.625,0.0,-0.230769,-0.095238
8,132539.0,2025-03-10 09:00:00,0.0,-1.060851,-2.575945,-0.596332,-0.401439,-1.591075,1.483758,1.301068,...,-0.735849,-0.176471,-0.309859,3.363636,0.0,0.0,-0.125,-2.0,-0.461538,-0.047619
9,132539.0,2025-03-10 10:00:00,0.0,-1.060851,-2.575945,-0.596332,-0.105285,-1.196678,0.064058,-0.814297,...,1.188679,0.058824,-0.352113,-0.363636,2.5,0.0,-1.125,-1.0,-0.230769,-0.095238


In [26]:
# Check if ICUType is a feature
print(df_final["ICUType"].value_counts())

ICUType
 0.0    66657
 0.5    49675
-0.5    41305
-1.0    25779
Name: count, dtype: int64


# Step 4 - Save DataFrames to data/processed

In [ ]:
# After this cell, data/processed should include the following files:
# - set_a_final.parquet
# - set_b_final.parquet
# - set_c_final.parquet